In [1]:
import os
import sys
import csv
import json
import moment
import pymysql
import datetime

import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

%matplotlib inline

## This script can generate the results data file required for plots and tables

Step 1. Set which outcome variable you want to generate data for.

In [2]:
# data_file_name = 'data/pandas_df_v2_2020-04-25.pkl'
# data_file_name = 'data/pandas_df_v3_2020-04-25.pkl'
data_file_name = 'data/pandas_df_v3-1_2020-04-25.pkl'

df = pd.read_pickle(data_file_name)
_version, date_retrieved = data_file_name.split('_df_')[1].split('.')[0].split('_')
print(_version, date_retrieved)

v3-1 2020-04-25


In [6]:
def propci(p, n):
    s = np.sqrt((p*(1-p)/n))
    return (p, p-1.96*s, p+1.96*s)

In [3]:
# NaN indicate people who were not covid positive
covidpos = df.dropna()

# remove patients who were intubated before they were diagnosed
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] >= 0]

# remove patients that died before they were diagnosed
print(sum(covidpos["days_to_death"] < 0))
covidpos = covidpos[covidpos["days_to_death"] >= 0]

# remove patients who were intubated or died more than 90 days later
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] < 90]
covidpos = covidpos[covidpos["days_to_death"] < 90]

covidpos['age_over_65'] = (covidpos['age'] > 65)+1-1

covidpos.describe()

121
12
0


,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,macula,compl_def,coagulation,...,refctrl,race_black,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,age_over_65
count,6.397000e+03,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,...,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000,6397.000000
mean,1.095011e+09,0.075191,19.607629,0.085040,19.901516,57.134029,0.497264,0.013756,0.000625,0.184305,...,0.113647,0.221510,0.023136,0.283727,0.280131,0.191496,0.339534,0.369079,0.291387,0.375020
std,1.319922e+08,0.263721,11.330646,0.278963,11.141045,19.877320,0.500032,0.116487,0.025000,0.387763,...,0.317407,0.415295,0.150347,0.450841,0.449098,0.393509,0.473588,0.482593,0.454437,0.484166
min,1.000011e+09,0.000000,0.000000,0.000000,0.000000,0.002738,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.004378e+09,0.000000,11.000000,0.000000,11.000000,41.478439,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.009221e+09,0.000000,20.000000,0.000000,20.000000,58.510609,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.200224e+09,0.000000,28.000000,0.000000,28.000000,72.005476,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.400038e+09,1.000000,84.000000,1.000000,83.000000,120.202601,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
covidpos[covidpos['intubated']==1].describe()

,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,macula,compl_def,coagulation,...,refctrl,race_black,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,age_over_65
count,4.810000e+02,481.0,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.0,481.000000,...,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000
mean,1.110304e+09,1.0,2.869023,0.347193,18.785863,62.241497,0.634096,0.029106,0.0,0.230769,...,0.158004,0.218295,0.008316,0.237006,0.303534,0.232848,0.482328,0.280665,0.237006,0.503119
std,1.579439e+08,0.0,3.747985,0.476574,11.567640,16.810648,0.482184,0.168279,0.0,0.421764,...,0.365125,0.413519,0.090907,0.425689,0.460263,0.423086,0.500208,0.449792,0.425689,0.500511
min,1.000051e+09,1.0,0.000000,0.000000,0.000000,0.084873,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.004109e+09,1.0,0.000000,0.000000,10.000000,53.023956,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.008096e+09,1.0,1.000000,0.000000,19.000000,65.048597,1.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.201186e+09,1.0,4.000000,1.000000,26.000000,73.237509,1.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,1.400038e+09,1.0,28.000000,1.000000,82.000000,94.132786,1.000000,1.000000,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
propci(0.347193, 481)

(0.347193, 0.30464673754986016, 0.3897392624501398)

In [8]:
covidpos[covidpos['died']==1].describe()

,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,macula,compl_def,coagulation,...,refctrl,race_black,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,age_over_65
count,5.440000e+02,544.000000,544.000000,544.0,544.000000,544.000000,544.000000,544.000000,544.0,544.000000,...,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000
mean,1.084413e+09,0.306985,15.845588,1.0,7.382353,76.839861,0.575368,0.040441,0.0,0.318015,...,0.143382,0.181985,0.009191,0.266544,0.312500,0.229779,0.479779,0.292279,0.227941,0.841912
std,1.420085e+08,0.461668,11.618794,0.0,6.994140,13.425636,0.494742,0.197173,0.0,0.466134,...,0.350785,0.386188,0.095517,0.442559,0.463939,0.421078,0.500051,0.455229,0.419891,0.365159
min,1.000051e+09,0.000000,0.000000,1.0,0.000000,6.272416,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.003690e+09,0.000000,4.750000,1.0,2.750000,70.364819,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.006066e+09,0.000000,17.000000,1.0,5.500000,78.197125,1.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.102049e+09,1.000000,24.000000,1.0,10.000000,86.937714,1.000000,0.000000,0.0,1.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,1.400037e+09,1.000000,84.000000,1.0,47.000000,102.373717,1.000000,1.000000,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
